## Llama Index and Llama 2 tutorial on Lonestar 6

Llama2 is the Meta open source Large Language Model. LlamaIndex is a python library that connects data to the LLMs such as Llama2. This allows the user to quickly use their unstructured data as a basis for any chats or outputs. 


In [1]:
from LLM_location import *
import logging
import sys

/scratch/06659/wmobley


In [2]:
%%html
<style>

.box_style{
    height: auto;
    background-color:green;
}
</style>

## Set your working directory
Change your working directory to your Scratch location. This will improve performance, and ensure you have access to the model you rsynced earlier

In [3]:
scratch = os.getenv('SCRATCH') 


## Access the model
Next we'll access the models. You have 4 models to access the 7 and 13billion parameters chat and normal model. The folder will also have access to the 70b parameter models; however, we have not tested their performance on the LS6 dev machines. 



## Load the Model

Next we'll load the model. If it can't find the model it will download it. 

In [4]:
engine = Story_Engine()

Accordion(children=(HBox(children=(Dropdown(description='Model:', index=3, options=('LLAMA2 7B', 'LLAMA2 7B CH…

## Load the PDF documents 

In [5]:
engine.create_index()


Error Creating Index: Please make sure you chosen the model and the Corpus you want to use for the anlaysis.


AttributeError: 'Story_Engine' object has no attribute 'index'

In [ ]:
engine.create_query_engine()

In [ ]:

engine.corpus.corpus_panel(engine.corpus.documents)

In [ ]:
response = engine.query("Who is the author?")
#

In [ ]:
import time
start = time.time()
response = query("What sampling approaches are used for estimating states of the world?")
print(( time.time()-start)/60)

In [ ]:
 query("How computationally intensive are DMDU Processes?")


In [ ]:
response = query("what makes robust decision making difficult to understand? Please cite sources")


In [ ]:
response = query("what is an integrated modeling platform? Please provide source bibliography")


In [ ]:
for i in range(len(response.source_nodes)):
    display(response.source_nodes[i].node.metadata)
    display(Markdown(response.source_nodes[i].node.get_text()))

In [ ]:
response = query("What are the limitations when applying robust decision making to a problem.")


In [ ]:
for i in range(len(response.source_nodes)):
    display(Markdown(response.source_nodes[i].node.get_text()));
    display(response.source_nodes[i].node.metadata);

In [ ]:
response = query("what are five features can improve user experience in  Decision making under deep uncertainty software, Please cite your sources")


In [ ]:
response = query("what features can reduce the complexity of the decision making under deep uncertainty process? Please cite your sources")


In [ ]:
response = query("Decision Making under deep uncertainty has a high learning curve, what could be added to a gui to reduce this learning curve? Please cite your sources")


In [ ]:
for i in range(len(response.source_nodes)):
    display(Markdown(response.source_nodes[i].node.get_text()));
    display(response.source_nodes[i].node.metadata);

In [ ]:
query("""What are the types of DMDU analyses used within the documents? """)


In [ ]:
query("""DMDU uses the following steps: 1) Problem Framing
	- Identify
		- objectives
		- constraints
		- major uncertainties
		- definition of success.
2) Identify when the status quo starts to fail. 
	- Simulate Business as Usual
	- Identify tipping points into a failing system
		- Need to identify rules for switching interventions
3) Identify and measure interventions
4) Explore Pathways from interventions
5) Design Adaptive Plan


Provide a list of triplets that identifies the algorithms used with each part.\n\n\
                          """)
